# Using APIs to Gather Data
### by [Jason DeBacker](http://jasondebacker.com), October 2017

This notebook provides a tutorial and examples showing how to use APIs and Python in order to gather data relevant for economic research.

## What is an API?

API stands for "Application Programming Interface".  APIs provide a way for one piece of software to interact with another by sharing data.  As such, APIs are very general and have many uses.  For example, APIs are used when you copy something from your internet browser and paste it into your word processor.  Here, we are going to focus on a relatively narrow application of APIs.  Specifically, we'll focus on using APIs to help use compile data that we can use for research.

## Why use an API?

A relevant question is why one would want to use an API rather than downloading a static datafile.  There are a few reasons for using APIs over static data:
1. Static datasets may not be available for the particular data source you are interesting in.
2. Static datasets may be too large to want to deal with - and your research involves just a selection of these data.
3. The data is changing quickly.

The last may be less relevant to academic research given our timelines, but the first two are certainly applicable.

## The JSON file format

Before we get started using APIs, lets discuss the [JSON (JavaScript Object Notation)](http://www.json.org) file format. JSON has become a standard data foramt and many APIs pass data as JSON files.  The JSON format is human-readable, but also easy for machines to parse and generate.  JSON is based on key-value pairs (like a Python dictionary) and ordered lists.  Here's an example of what JSON data look like:

```
{
     "firstName": "John",
     "lastName": "Smith",
     "address": {
         "streetAddress": "21 2nd Street",
         "city": "New York",
         "state": "NY",
         "postalCode": 10021
     },
     "phoneNumbers": [
         "212 555-1234",
         "646 555-4567"
     ]
 }
```

You can see some similarities between the what JSON is written and the way one writes data to a Python dictioinary.  They contain similar structures.  Another thing to notice about the JSON format - you can clearly understand the data that are being represented here and what they mean.

## Example 1: Using `pandas-datareader`

One of the easiest ways to use an API is Python is to use the `pandas-datareader` package.  Fortunately for our purposes, this package also provides us the ability to connect to APIs for a number of datasources that are commonly used in economics such as:

* Quandl
* St.Louis FED (FRED)
* Kenneth French’s data library
* World Bank
* OECD
* Yahoo! Finance
* Google Finance
* Eurostat
* Enigma

For the example that follows, we will gather some data from FRED and plot it.  `pandas-datareader` provides a wrapper for the [FRED API](https://research.stlouisfed.org/docs/api/fred/).


In [ ]:
# import pandas-datareader and other packages
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline  

# set beginning and end dates for data
start = datetime.datetime(1947, 1, 1) # format is year (1947), month (1), day (1)
end = datetime.date.today() # go through today

# pull series of interest using pandas_datareader
fred_data=web.DataReader(["USREC", "GDPC1", "GPDIC96"], "fred", start, end)
fred_data.head(n=10)

In [ ]:
# some data are monthly - put everything into quartely by averaging over months
fred_data = fred_data.resample('Q').mean()
fred_data = fred_data[:-1] # drop last quarter since not all data available

# First-difference and detrend the data
# Note: probably a better way to do this
fred_data['dt_gdp']=fred_data['GDPC1'].pct_change(freq='Q') - np.mean(fred_data['GDPC1'].pct_change(freq='Q'))
fred_data['dt_inv']=fred_data['GPDIC96'].pct_change(freq='Q') - np.mean(fred_data['GPDIC96'].pct_change(freq='Q'))

# plot GDP and investment series
ax = fred_data.plot(y=['dt_gdp', 'dt_inv'], title="Volatility of GDP and Investment",
               grid=True, lw=1, color = ['blue', 'red'])
# Label y-axis
ax.set_ylabel("Percent Change from Previous Quarter")
# rename column vars for legend
ax.legend(['GDP', 'Investment'])
# add a zero line
ax.axhline(0, color='k', linestyle='-', linewidth=1)
# shade recessions
ax.fill_between(fred_data.index, ax.get_ylim()[0], ax.get_ylim()[1], where=fred_data['USREC'].values, color='gray', alpha=0.2)

This is pretty neat.  We can, with few line of code, gather and plot any data that FRED has.  `pandas-datareader` is doing a lot behind the scenes.  It is contacting the web address of the FRED servers which store these data, making a request to get these data, downloading those JSON files, and then creating the `fred_data` dataframe object from those JSON files. 

## Example 2:  Accessing Trade Data

Now we'll use an API for which there is not a Python wrapper.  In particular, we'll download trade data from the Observatory of Economic Complexity (OEC).  The OEC has compiled trade data from sources such as the United Nations Statistical Division (COMTRADE) and put them in one place.  These databases are large - trade flows by very dissaggregated levels of goods between all most country pairs from 1962-2015.

Fortunately, the OEC has an API we can use to access these data.  Details on how to use the API are [here](http://atlas.media.mit.edu/api/).  Let's use this and look at some trends in trade flows between the U.S. and China.

In [ ]:
import requests
import json

# Format for URL: / CLASSIFICATION / TRADE_FLOW / YEAR / ORIGIN / DESTINATION / PRODUCT /
url = "http://atlas.media.mit.edu/sitc/export/1962.2015/chn/usa/all/"
response = requests.get(url)
trade_data = response.text
trade_data

In [ ]:
df = pd.read_json(trade_data, orient='split')

In [ ]:
df.describe()

In [ ]:
# plot data
plt.style.use('seaborn')
# use data just for one SITC category - Rubber tires, SITC code 106251
df[df['sitc_id']==106251].plot(x='year', y=['import_val','export_val'], kind='bar',
                              title='Chinese imports and exports by year - Rubber tires')

## Example 3: Twitter data

### Setting up

To use the Twitter API, you will need to have some credentials to authenticate yourself.  You can set these up by going to [https://apps.twitter.com](https://apps.twitter.com) and creating a new app.  This will allow you to get the credentials you need: `Consumer Key` and `Consumer Secret`. Depending on what you are doing, you might also need the `Access Token Key` and the `Access Token Secret`.  Grab all of these and store them in a file THAT YOU DO NOT SHARE (these are private).  I put these in a JSON file called `credentials.json`.  It contains the follwing (with my credentials removed):

```
{
  "consumer_key" : "...",
  "consumer_secret" : "...",
  "access_token_key" : "...",
  "access_token_secret" : "..."
}

```

The Twitter API has some notable limitations.  For example, the twitter API limits your search results to 100 tweets (in many circumstances, some times you can get more, but still not many thousands).  You can get more tweets by opening up a streaming API to gather tweets in realtime.  This can be done with they `TwythonStreaming` module - or with other packages that interface with the Twitter API.  

Note there are also Twitter APIs for enterprise use that allow you to gather more data, but they cost $$.

### Searching for tweets with a specific query

First, let's see how to grab tweets with a query.  E.g., we can try to grab tweets with a certain phrase in them.  There are seveal Python packages that provide wrappers or the Twitter API.  We'll use [Twython](https://twython.readthedocs.io/en/latest/).  [Tweepy](https://github.com/tweepy/tweepy) seems to also be popular.  But they all have similar structure, so once you get the handle on one, you can probably use others without much of a learning curve.

In [ ]:
# import packages
from twython import Twython
from twython.exceptions import TwythonError
import json

# import credentials to connect to Twitter API
with open('credentials.json') as f:
    creds = json.loads(f.read())

# Initiate the client
client = Twython(
    creds['consumer_key'],
    creds['consumer_secret'],
    creds['access_token_key'],
    creds['access_token_secret']
)

In [ ]:
# specify the search terms - and filter out retweets
query = '#MAGA -filter:retweets'

# use the client to run a search
# geocode = '25.032341,55.385557,100000mi' # latitude,longitude,distance(mi/km)

results = client.search(q=query, count=100)
# results = client.search(q=query, count=100, geocode=geocode)

In [ ]:
# Look at how the results are structured
results.keys()

In [ ]:
len(results['statuses'])

In [ ]:
# take a loot at data from a single tweet
results['statuses'][0]

In [ ]:
# put results in a DataFrame
df = pd.DataFrame(results['statuses'])

### Searching for tweets from a specific user

Another option we have with Twython is to search for results from a specific user.  Here we need to provide a screen name or user id.  We can return up to about the 200 most recent tweets from the user.

In [ ]:
tweets = []
tweet_loc = []
try:
    user_timeline = client.get_user_timeline(screen_name='realdonaldtrump',count=200)
except TwythonError as e:
    print(e)
print(len(user_timeline))
for tweet in user_timeline:
    # Add whatever you want from the tweet, here we just add the text
    tweets.append(tweet['text'])
    tweet_loc.append(tweet['geo'])
# Count could be less than 200, see:
# https://dev.twitter.com/discussions/7513
while len(user_timeline) != 0: 
    try:
        user_timeline = client.get_user_timeline(screen_name='realdonaltrump',
                                                 count=500,max_id=user_timeline[len(user_timeline)-1]['id']-1)
    except TwythonError as e:
        print(e)
    print(len(user_timeline))
    for tweet in user_timeline:
        # Add whatever you want from the tweet, here we just add the text
        tweets.append(tweet['text'])
        tweet_loc.append(tweet['geo'])
# Number of tweets the user has made
print(len(tweets))

In [ ]:
# Looking at what is in the tweets we've compiled
tweets[0]

In [ ]:
# Each tweet has much more infor than just the text we saved
tweet.keys()

In [ ]:
tweet

In [ ]:
# Put the tweets in a dataframe
trump_df = pd.DataFrame(tweets)
trump_df

In [ ]:
# Plot the frequency of certain text
trump_df['Great'] = trump_df[0].str.contains('Great', case=False)
trump_df['Trump'] = trump_df[0].str.contains('Trump')
trump_df['America'] = trump_df[0].str.contains('America')
trump_df['Tax'] = trump_df[0].str.contains('Tax', case=False)
trump_df[["Great", "NFL", "Trump", "America", "Tax"]].sum().plot(kind = "bar", title='Trump Tweet Topics')